In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [6]:
df = pd.read_csv('masterfile-2024.csv', sep=';')

In [13]:
def classify_non_maint(df, CH_threshold=9, SQ_ratio_threshold=0.05, SQ_time_threshold=70):
    """
    Classifies each file as 'maintainable' or 'not maintainable' based on the code_health score.
    The classification result is stored in a new column 'maintainability' within the same DataFrame.
    
    Parameters:
    - df: Pandas DataFrame containing the files data.
    - threshold: The code_health score threshold to consider a file as maintainable.
    
    Returns:
    - The DataFrame with an additional column 'maintainability' indicating the classification.
    """
    # Convert columns to numerica, ensuring proper comparison
    df['code_health'] = pd.to_numeric(df['code_health'], errors='coerce')
    df['TD_ratio'] = pd.to_numeric(df['TD_ratio'], errors='coerce')
    
    # Apply classification based on 'code_health' score and threshold
    df[str('non-green')] = df['code_health'].apply(lambda x: True if x < CH_threshold else False)
    
    # Apply classification based on 'TD_ratio' score and threshold
    df[str('non-indexA')] = df['TD_ratio'].apply(lambda x: True if x >= SQ_ratio_threshold else False)

    # Apply classification based on 'TD_ratio' score and threshold
    df[str('non-quick')] = df['TD_time'].apply(lambda x: True if x >= SQ_time_threshold else False)
    
    return df

In [25]:
classify_non_maint(df)
df

,projectname,packageandclass,path,Readability,Understandability,Complexity_inv,Modularization_inv,Overall,codehealth_old,code_health,TD_time,TD_ratio,non-green,non-indexA,non-quick
0,aoi,artofillusion.animation.ActorEditorWindow,aoi\sourcefiles\ArtOfIllusion\src\artofillusio...,1,2,2,1,2,9.15,8.99,242,0.022,True,False,True
1,aoi,artofillusion.animation.AnimationPreviewer,aoi\sourcefiles\ArtOfIllusion\src\artofillusio...,2,2,1,1,3,8.89,8.73,354,0.047,True,False,True
2,aoi,artofillusion.animation.distortion.CustomDisto...,aoi\sourcefiles\ArtOfIllusion\src\artofillusio...,1,3,1,1,2,9.15,8.96,25,0.009,True,False,False
3,aoi,artofillusion.animation.FilterParameterTrack,aoi\sourcefiles\ArtOfIllusion\src\artofillusio...,1,2,1,0,2,9.37,9.24,144,0.016,False,False,True
4,aoi,artofillusion.animation.IKTrack,aoi\sourcefiles\ArtOfIllusion\src\artofillusio...,1,2,1,0,2,9.87,9.84,264,0.024,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,junit4,org.junit.runners.parameterized.BlockJUnit4Cla...,junit4\sourcefiles\src\main\java\org\junit\run...,1,1,2,2,1,9.87,9.87,105,0.028,False,False,True
300,junit4,org.junit.runners.parameterized.BlockJUnit4Cla...,junit4\sourcefiles\src\main\java\org\junit\run...,0,0,3,3,0,10.00,10.00,0,0.000,False,False,False
301,junit4,org.junit.runners.parameterized.TestWithParame...,junit4\sourcefiles\src\main\java\org\junit\run...,0,0,3,3,0,10.00,10.00,0,0.000,False,False,False
302,junit4,org.junit.validator.AnnotationsValidator,junit4\sourcefiles\src\main\java\org\junit\val...,1,1,2,2,1,10.00,10.00,7,0.003,False,False,False


In [15]:
def calculate_metrics(df, pred_column):
    """
    Calculate and print out statistics for a given prediction column.
    
    Parameters:
    - df: DataFrame with your data
    - pred_column: Column name for the prediction results
    """
    
    # Calculate True Positives (TP), False Positives (FP), True Negatives (TN), and False Negatives (FN)
    TP = ((df[pred_column] == True) & (df['Overall'] >= 2)).sum()
    FP = ((df[pred_column] == True) & (df['Overall'] <= 1)).sum()
    TN = ((df[pred_column] == False) & (df['Overall'] <= 1)).sum()
    FN = ((df[pred_column] == False) & (df['Overall'] >= 2)).sum()
    
    # Calculate accuracy, precision, and recall
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0

    # Calculate F1 Score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Calculate F2 Score - gives more weight to recall
    f2 = (5 * precision * recall) / ((4 * precision) + recall) if (precision + recall) > 0 else 0

    # Calculate F0.5 Score - gives more weight to precision
    f0_5 = (1.25 * precision * recall) / ((0.25 * precision) + recall) if (precision + recall) > 0 else 0
    
    # Print the metrics
    print(f"Metrics for {pred_column}:")
    print(f"TP: {TP}")
    print(f"TN: {TN}")
    print(f"FP: {FP}")
    print(f"FN: {FN}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")  
    print(f"F2-score: {f2:.4f}")
    print(f"F0.5-score: {f0_5:.4f}")

In [26]:
calculate_metrics(df, 'non-green')

Metrics for non-green:
TP: 51
TN: 232
FP: 6
FN: 15
Accuracy: 0.9309
Precision: 0.8947
Recall: 0.7727
F1-score: 0.8293
F2-score: 0.7944
F0.5-score: 0.8673


In [27]:
calculate_metrics(df, 'non-indexA')

Metrics for non-indexA:
TP: 8
TN: 176
FP: 62
FN: 58
Accuracy: 0.6053
Precision: 0.1143
Recall: 0.1212
F1-score: 0.1176
F2-score: 0.1198
F0.5-score: 0.1156


In [28]:
calculate_metrics(df, 'non-quick')

Metrics for non-quick:
TP: 52
TN: 191
FP: 47
FN: 14
Accuracy: 0.7993
Precision: 0.5253
Recall: 0.7879
F1-score: 0.6303
F2-score: 0.7163
F0.5-score: 0.5628
